In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import math
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler, QuantileTransformer
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from imblearn.over_sampling import SMOTE
import scipy.stats as stats
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import random
from deap import base, creator, tools, algorithms
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import random
from deap import base, creator, tools, algorithms
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score


# XGBoost

In [ ]:
import random
from deap import base, creator, tools, algorithms
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from xgboost import XGBClassifier

# (Optional) matplotlib
try:
    import matplotlib.pyplot as plt
    has_matplotlib = True
except ImportError:
    has_matplotlib = False


# ------------------------------------------------
# 1) DEAP의 creator 중복 생성 방지
# ------------------------------------------------
def create_deap_classes():
    try:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    except AttributeError:
        pass  # 이미 생성되었으면 무시

    try:
        creator.create("Individual", list, fitness=creator.FitnessMax)
    except AttributeError:
        pass  # 이미 생성되었으면 무시

create_deap_classes()

toolbox = base.Toolbox()

# ------------------------------------------------
# 2) 개체(Individual) 생성
#    - 각 특징을 선택(1) / 제외(0)
# ------------------------------------------------
toolbox.register("attr_bool", random.randint, 0, 1)

def initialize_individual():
    return tools.initRepeat(creator.Individual, toolbox.attr_bool, n=X.shape[1])

toolbox.register("individual", initialize_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# ------------------------------------------------
# 3) GA 평가함수: XGB + GridSearchCV
# ------------------------------------------------
def evalFeatureSelection(individual, X, y, cv_folds=4):
    """
    individual: 0/1로 구성된 feature mask
    X, y: 전체 데이터 (numpy array 등)
    cv_folds: StratifiedKFold 폴드 수
    """
    # 선택된 피처 인덱스 추출
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    # 아무 피처도 선택되지 않은 경우
    if len(selected_features) == 0:
        return (0.0,)

    # XGBoost 하이퍼파라미터 그리드 (예시)
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [3, 5],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    xgb_model = XGBClassifier(
        objective='multi:softprob',
        eval_metric='mlogloss',
        random_state=42,
    )

    # StratifiedKFold 정의
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

    # f1_macro를 목표로 GridSearch
    scoring = make_scorer(f1_score, average='weighted')
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring=scoring,
        cv=skf,
        n_jobs=-1,
        verbose=0
    )

    # 선택된 피처만으로 학습
    grid_search.fit(X[:, selected_features], y)

    # best_score_는 폴드 평균 f1_macro
    return (grid_search.best_score_,)


toolbox.register("evaluate", evalFeatureSelection, X=X, y=y)

# ------------------------------------------------
# 4) 유전 연산 등록 (교차, 돌연변이, 선택)
# ------------------------------------------------
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.02)  # mutation probability per gene
toolbox.register("select", tools.selTournament, tournsize=3)


# ------------------------------------------------
# 5) 메인 함수: GA 실행 + 최종 평가
# ------------------------------------------------
def main():
    create_deap_classes()
    random.seed(42)

    # GA 초기 설정
    population_size = 20  # 초기 인구 수
    ngen = 10             # 세대 수
    cxpb = 0.7            # 교차 확률
    mutpb = 0.1           # 돌연변이 확률

    # Early Stopping
    patience = 3
    best_fitness = -np.inf
    no_improvement = 0

    # 초기 인구 생성
    population = toolbox.population(n=population_size)

    best_fitness_per_gen = []
    avg_fitness_per_gen = []

    for gen in range(ngen):
        # 1) 자손 생성 (교차 + 돌연변이)
        offspring = algorithms.varAnd(population, toolbox, cxpb, mutpb)
        
        # 2) 각 자손 피트니스 평가 (GridSearchCV 수행)
        fits = list(map(toolbox.evaluate, offspring))
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit

        # 3) 다음 세대 선택
        population = toolbox.select(offspring, k=len(population))

        # 4) 현재 세대의 Best
        best_ind = tools.selBest(population, k=1)[0]
        current_best_fitness = best_ind.fitness.values[0]

        best_fitness_per_gen.append(current_best_fitness)
        avg_fitness = np.mean([ind.fitness.values[0] for ind in population])
        avg_fitness_per_gen.append(avg_fitness)
        print(f"[Generation {gen+1}] Best F1(macro) = {current_best_fitness:.4f}, Avg F1(macro) = {avg_fitness:.4f}")

        # Early Stopping 체크
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print(f"\nEarly Stopping triggered at generation {gen+1}.")
            break

    # (Optional) Fitness 진화 과정 시각화
    if has_matplotlib:
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(best_fitness_per_gen)+1), best_fitness_per_gen, 
                 label='Best F1(macro)')
        plt.plot(range(1, len(avg_fitness_per_gen)+1), avg_fitness_per_gen, 
                 label='Average F1(macro)')
        plt.xlabel('Generation')
        plt.ylabel('F1 Score (macro)')
        plt.title('Fitness Evolution with XGBoost + GridSearchCV')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    # 최종 선정된 특징 (best_ind)
    best_ind = tools.selBest(population, k=1)[0]
    selected_features = [i for i, bit in enumerate(best_ind) if bit == 1]
    print("\n--- 최종 선정된 특성 인덱스 ---")
    print(selected_features)
    print(f"Selected Feature Count: {len(selected_features)}")

    if len(selected_features) == 0:
        print("선택된 특성이 없습니다. 성능 측정 불가.")
        return

    # 최종 모델 학습/평가 (원하면 한 번 더 GridSearchCV 가능)
    # 여기서는 간단히 "best_ind" 피처만 사용해 모델 성능 측정
    final_xgb = XGBClassifier(
        objective='multi:softprob',
        eval_metric='mlogloss',
        random_state=42,
    )

    # 5-Fold 교차검증
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy_list = []
    f1_list = []

    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X[:, selected_features], y), 1):
        X_train, X_test = X[train_idx][:, selected_features], X[test_idx][:, selected_features]
        y_train, y_test = y[train_idx], y[test_idx]

        final_xgb.fit(X_train, y_train)
        y_pred = final_xgb.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        f1_m = f1_score(y_test, y_pred, average='weighted')
        accuracy_list.append(acc)
        f1_list.append(f1_m)
        print(f"Fold {fold_idx} => Accuracy: {acc:.4f}, F1: {f1_m:.4f}")

    final_accuracy = np.mean(accuracy_list)
    final_f1 = np.mean(f1_list)
    print(f"\n[최종 교차 검증 결과] Accuracy: {final_accuracy:.4f}, F1: {final_f1:.4f}")


if __name__ == "__main__":
    main()


# CatBoost

In [7]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, make_scorer, accuracy_score
from catboost import CatBoostClassifier


def create_deap_classes():
    try:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    except AttributeError:
        pass

    try:
        creator.create("Individual", list, fitness=creator.FitnessMax)
    except AttributeError:
        pass

create_deap_classes()

toolbox = base.Toolbox()


# ------------------------------------------------
# 3) 개체(Individual) 생성: 0/1 Feature Mask
# ------------------------------------------------
toolbox.register("attr_bool", random.randint, 0, 1)

def initialize_individual():
    return tools.initRepeat(creator.Individual, toolbox.attr_bool, n=X.shape[1])

toolbox.register("individual", initialize_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# ------------------------------------------------
# 4) GA 평가함수: CatBoost + GridSearchCV (Weighted F1)
# ------------------------------------------------
def evalFeatureSelection(individual, X, y, seed, cv_folds=4):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:
        return (0.0,)  # 아무 특성도 없으면 점수 0

    param_grid = {
        'iterations': [100, 200],
        'depth': [3, 5],
        'learning_rate': [0.01, 0.1],
        'l2_leaf_reg': [1, 5],
        'rsm': [0.8, 1.0]
    }

    # CatBoost 모델
    cat_model = CatBoostClassifier(
        loss_function='MultiClass',
        random_seed= seed,
        verbose=0
    )

    # 교차검증/스코어링 설정
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    scoring = make_scorer(f1_score, average='weighted')

    grid_search = GridSearchCV(
        estimator=cat_model,
        param_grid=param_grid,
        scoring=scoring,
        cv=skf,
        n_jobs=-1,
        verbose=0
    )

    # 선택된 피처만으로 학습 + 튜닝
    grid_search.fit(X[:, selected_features], y)

    # grid_search.best_score_ => 교차검증 평균 Weighted F1
    return (grid_search.best_score_,)



def catboost(X, y, seed):
    create_deap_classes()
    
    toolbox.register("evaluate", evalFeatureSelection, X=X, y=y, seed=seed)


    # ------------------------------------------------
    # 5) 유전 연산 (교차, 돌연변이, 선택) 등록
    # ------------------------------------------------
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.02)
    toolbox.register("select", tools.selTournament, tournsize=3)

    random.seed(seed)

    # GA 파라미터
    population_size = 20   # 초기 인구
    ngen = 10              # 세대 수
    cxpb = 0.7             # 교차 확률
    mutpb = 0.1            # 돌연변이 확률

    # Early Stopping
    patience = 3
    best_fitness = -np.inf
    no_improvement = 0

    # 1) 초기 인구 생성
    population = toolbox.population(n=population_size)

    best_fitness_per_gen = []
    avg_fitness_per_gen = []

    for gen in range(ngen):
        # (a) 자손 생성 (교차+돌연변이)
        offspring = algorithms.varAnd(population, toolbox, cxpb, mutpb)
        
        # (b) 각 자손의 피트니스 계산 (CatBoost + GridSearchCV)
        fits = list(map(toolbox.evaluate, offspring))
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit

        # (c) 다음 세대 선택
        population = toolbox.select(offspring, k=len(population))

        # (d) 현재 세대 Best
        best_ind = tools.selBest(population, k=1)[0]
        current_best_fitness = best_ind.fitness.values[0]

        best_fitness_per_gen.append(current_best_fitness)
        avg_fitness = np.mean([ind.fitness.values[0] for ind in population])
        avg_fitness_per_gen.append(avg_fitness)

        print(f"[Generation {gen+1}] Best F1(weighted) = {current_best_fitness:.4f}, Avg F1 = {avg_fitness:.4f}")

        # Early Stopping 여부 체크
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print(f"\nEarly Stopping triggered at generation {gen+1}.")
            break


    best_ind = tools.selBest(population, k=1)[0]
    selected_features = [i for i, bit in enumerate(best_ind) if bit == 1]
    print("\n--- 최종 선정된 특성 인덱스 ---")
    print(selected_features)
    print(f"Selected Feature Count: {len(selected_features)}")

    if len(selected_features) == 0:
        print("선택된 특성이 없습니다. 성능 측정 불가.")
        return

    # 최종 평가(간단하게 CatBoost 기본 설정 + 5-Fold CV)
    final_cat = CatBoostClassifier(loss_function='MultiClass', random_seed=42, verbose=0)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracy_list = []
    f1_list = []

    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X[:, selected_features], y), 1):
        X_train, X_test = X[train_idx][:, selected_features], X[test_idx][:, selected_features]
        y_train, y_test = y[train_idx], y[test_idx]

        final_cat.fit(X_train, y_train)
        y_pred = final_cat.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        f1_w = f1_score(y_test, y_pred, average='weighted')
        accuracy_list.append(acc)
        f1_list.append(f1_w)
        print(f"Fold {fold_idx} => Accuracy: {acc:.4f}, F1(weighted): {f1_w:.4f}")

    final_acc = np.mean(accuracy_list)
    final_f1 = np.mean(f1_list)
    print(f"\n[최종 교차 검증 결과] Accuracy: {final_acc:.4f}, F1(weighted): {final_f1:.4f}")

    return final_f1

c:\Users\carly\anaconda3\envs\dyspepsia_research\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\carly\anaconda3\envs\dyspepsia_research\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
dataset_link = [
    "../dataset/v2_seperate/mec.csv",
    "../dataset/v2_seperate/skin.csv",
    "../dataset/v2_seperate/survey.csv",
    "../dataset/v2_seperate/v2_eda.csv",
    "../dataset/v2_seperate/v2_original.csv"
]

for link in dataset_link:
    df = pd.read_csv(link)
    y = df['diagnosis_type'].values #diagnosis : 6class, diagnosis_type : 3 class

    df.drop(columns=['No', 'diagnosis', 'diagnosis_type'], inplace=True)

    X = df.values
    y = y - 1
    
    
    ####Genetic-RandomForest######################################
    ##################################################
    ##################################################
    seed_li = [1557, 2356, 88488]
    result = 0
    
    for seed in seed_li:
        result += catboost(X, y, seed)
    
    result = result / 3
    print(link, result)

SyntaxError: invalid syntax (740854374.py, line 28)